In [1]:
import pandas as pd
import numpy as np

# Stars.csv

In [2]:
clusters = ["pleiades","hyades", "praesepe",  "ngc6774", "ngc2682"]

In [3]:
columns = ["EPIC","todrop","Teff_median", "Teff_std", 
           "Lum_SED","e_Lum_SED",
           "Lum_Kepler", "e_Lum_Kepler", "Campaign",           
           "Rstar", "e_Rstar", "PMem_mean", "RAJ2000_K2", 
           "DEJ2000_K2", "cluster"]
dfs = pd.DataFrame(columns=columns)
nrows = 0
for cluster in clusters:
    df = pd.read_csv(f"{cluster}_luminosities.csv",usecols=columns[:-1])
    df["cluster"] = cluster
    nrows += df.shape[0]
    dfs = pd.concat([dfs, df], ignore_index=True)
    
dfs = dfs.drop_duplicates()    
dfs.shape

(3237, 15)

In [4]:
flares = pd.read_csv("../../k2scoc/results/tables/2020_05_final_flares.csv")
wassearched = (flares.real==0)  & (flares.todrop.isnull())
hasflares = (flares.real==1) & (flares.todrop.isnull())
Teffs = (flares.Teff_median>2500)  & (flares.Teff_median < 6000)
flares = flares[(wassearched | hasflares) & Teffs]

In [5]:
epics = flares.EPIC.drop_duplicates()
epics.shape

(2111,)

In [6]:
dfs2 = dfs[dfs.EPIC.isin(epics)]
dfs2 = dfs2.drop_duplicates(subset=['EPIC', 'todrop', 'Teff_median', 'Teff_std', 'Lum_Kepler',
       'e_Lum_Kepler', 'Campaign', 'Rstar', 'e_Rstar', "Lum_SED","e_Lum_SED",
       'RAJ2000_K2', 'DEJ2000_K2', 'cluster'])
dfs2.shape

(2111, 15)

In [7]:
dfs2.EPIC = dfs2.EPIC.astype(int)
dfs2.Rstar = np.round(dfs2.Rstar,2).fillna("-999")
dfs2.e_Rstar = np.round(dfs2.e_Rstar,2).fillna(-999)
dfs2.PMem_mean = np.round(dfs2.PMem_mean,2).fillna(-999)
dfs2.Lum_Kepler = dfs2.Lum_Kepler.fillna(-999)
dfs2.e_Lum_Kepler = dfs2.e_Lum_Kepler.fillna(-999)
#dfs2.Campaign = np.rint(dfs2.Campaign).fillna(extra_campaigns.setdefault(cluster, -999)).astype(int)
dfs2.Teff_median = np.rint(dfs2.Teff_median).fillna(-999).astype(int)
dfs2.Teff_std = np.rint(dfs2.Teff_std).fillna(-999).astype(int)
#dfs2.todrop = dfs2.todrop.fillna("-999").str.replace(" outlier,",";")
dfr = dfs2.rename(index=str, columns={"EPIC":"ID",
                                   "Teff_median":"Teff",
                                   "Teff_std":"e_Teff",
                                   "Rstar":"R",
                                   "e_Rstar":"e_R",
                                   "PMem_mean":"pmem",
                                    "RAJ2000_K2":"RAJ2000",
                                    "DEJ2000_K2":"DEJ2000",
                                    "Lum_SED":"Lum_bol",
                                    "e_Lum_SED":"e_Lum_bol",
                                   })

In [8]:
dfr[["ID","Campaign", "RAJ2000", "DEJ2000", 
     "pmem",  "Teff", "e_Teff",
     "R", "e_R", "Lum_bol", "e_Lum_bol",
     "Lum_Kepler", "e_Lum_Kepler",
     ]].to_csv("../../CDS/stars.csv", index=False)

# Flares.csv

In [9]:
flares = pd.read_csv("../../k2scoc/results/tables/2020_05_final_flares.csv")
wassearched = (flares.real==0)  & (flares.todrop.isnull())
hasflares = (flares.real==1) & (flares.todrop.isnull())
Teffs = (flares.Teff_median>2500)  & (flares.Teff_median < 6000)
flares = flares[hasflares & Teffs]
flares.shape

(3844, 25)

In [10]:
flares2 = flares[['cstart', 'cstop', 'tstart', 'tstop', 'ed_rec',
       'ed_rec_err', 'ampl_rec', 'dtp_tot', 'EPIC', 'C','note', 'cluster', 'Lum_Kepler', 'e_Lum_Kepler', 'Teff_median',
       'Teff_std', ]]


In [11]:
flares2 = flares2.rename(index=str, columns={"EPIC":"ID",
                                
                                    "dtp_tot":"dtp",
                                    "ed_rec":"ED",
                                    "ed_rec_err":"e_ED",         
                                   })

In [12]:
flares2 = flares2.merge(dfr[["ID","Lum_bol","e_Lum_bol"]], how="left", on="ID")

In [13]:
cs = ['ID', 'C','cstart', 'cstop', 'tstart', 'tstop', 'ED', 'e_ED', 'ampl_rec', 
        'note', 'dtp','Lum_Kepler', 'e_Lum_Kepler',
       'Teff_median', 'Teff_std', 'Lum_bol', 'e_Lum_bol','cluster']
flares2 = flares2[cs]

In [14]:
for c in cs:
    flares2[c] = flares2[c].fillna(-999)

In [15]:
flares2

,ID,C,cstart,cstop,tstart,tstop,ED,e_ED,ampl_rec,note,dtp,Lum_Kepler,e_Lum_Kepler,Teff_median,Teff_std,Lum_bol,e_Lum_bol,cluster
0,211107267,4,105695.0,105703.0,2268.662766,2268.826219,11162.187772,105.663640,1.275359,-999,3282.0,2.754924e+29,1.501074e+29,3067.420021,126.456062,6.912767e+29,3.766356e+29,pleiades
1,246625561,13,144350.0,144353.0,3058.520378,3058.581673,2040.660840,82.610589,0.671565,-999,3643.0,3.077061e+30,1.274619e+30,3467.408865,132.350146,7.295500e+30,3.021832e+30,hyades
2,247122957,13,142526.0,142534.0,3021.252956,3021.416409,207.840120,8.012327,0.034262,-999,3643.0,5.049071e+30,8.236781e+29,3568.371626,75.123388,1.187343e+31,1.936725e+30,hyades
3,211973504,18,163090.0,163093.0,3441.450569,3441.511865,671.606663,12.190816,0.261612,-999,2371.0,1.502303e+30,5.383327e+29,3321.382531,88.591286,3.633734e+30,1.302046e+30,praesepe
4,211027393,4,104599.0,104604.0,2246.269609,2246.371769,28.245788,2.242767,0.005483,-999,2983.0,7.274280e+31,1.172526e+31,4918.958422,184.446799,1.621786e+32,2.613548e+31,pleiades
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3839,212128175,18,163887.0,163894.0,3457.734867,3457.877888,56.171613,3.334049,0.006776,-999,2371.0,1.084073e+31,1.182384e+30,3837.514660,59.091339,2.485062e+31,2.710057e+30,praesepe
3840,211901458,16,154749.0,154752.0,3271.009848,3271.071155,189.003902,13.652953,0.055730,-999,3346.0,6.919473e+29,2.611866e+29,3187.217095,83.121132,1.705295e+30,6.436639e+29,praesepe
3841,211924546,5,108947.0,108955.0,2335.113110,2335.276563,55.193959,1.704252,0.007885,-999,3147.0,1.069313e+31,1.166350e+30,3828.475436,62.755896,2.452477e+31,2.674618e+30,praesepe
3842,211924546,5,110020.0,110024.0,2357.036265,2357.117991,55.537024,0.923505,0.011631,-999,3147.0,1.069313e+31,1.166350e+30,3828.475436,62.755896,2.452477e+31,2.674618e+30,praesepe


In [16]:
flares2.to_csv("../../CDS/flares.csv", index=False)

# ffds.csv

In [17]:
ffds = pd.read_csv("../../VettingFlares/tables/cluster_SpT_alphabeta.csv")

In [18]:
cs = ['Teff', 'cluster', 'alpha', 'alpha_up_err', 'alpha_low_err', 'beta',
       'beta_up_err', 'beta_low_err', 'nflares', 'nfit', 'ED_min', 'ED_max',
       'alpha_en', 'alpha_up_err_en', 'alpha_low_err_en', 'beta_en',
       'beta_up_err_en', 'beta_low_err_en', 'nflares_en', 'nfit_en',
       'EKp_min', 'EKp_max', ]


In [19]:
ffds2 = ffds[cs]
ffds2 = ffds2.rename(index=str, columns={"alpha":"alpha_s",
                                         'alpha_up_err':"alpha_s_up_err", 
                                         'alpha_low_err':"alpha_s_low_err",
                                         'beta':"beta_s",
                                         'beta_up_err':"beta_s_up_err", 
                                         'beta_low_err':'beta_s_low_err', 
                                         'nflares':'nflares_s', 
                                         'nfit':'nfit_s',
                                         "alpha_en":"alpha_erg",
                                         'alpha_up_err_en':"alpha_erg_up_err", 
                                         'alpha_low_err_en':"alpha_erg_low_err",
                                         'beta_en':"beta_erg",
                                         'beta_up_err_en':"beta_erg_up_err", 
                                         'beta_low_err_en':'beta_erg_low_err', 
                                         'nflares_en':'nflares_erg', 
                                         'nfit_en':'nfit_erg',
                                   })

In [20]:
ffds2.head().T

,0,1,2,3,4
Teff,2500-3000,2500-3000,2500-3000,3000-3250,3000-3250
cluster,Pleiades,Hyades,Praesepe,Pleiades,Hyades
alpha_s,2.09905,2.09736,2.09866,2.08877,2.08084
alpha_s_up_err,0.0344902,0.0345154,0.0347142,0.0305873,0.0336322
alpha_s_low_err,0.0342464,0.0346872,0.0344695,0.0308793,0.033327
beta_s,2803.08,8102.43,2427.25,4842.45,2618.47
beta_s_up_err,1290.54,4201.76,1588.06,1262.6,709.176
beta_s_low_err,916.024,2853.76,1043.37,1011.88,570.973
nflares_s,23,26,9,616,137
nfit_s,12,9,4,258,76


In [21]:
ffds2.to_csv("../../CDS/ffds.csv", index=False)